# EfficientNet backbone

In [ ]:
import numpy as np

In [ ]:
from DatasetLoader import DatasetLoader

In [ ]:
datasetloader = DatasetLoader(dataset_root='../dataset/', img_resize=(224,224))

In [ ]:
images, labels = datasetloader.loadTrain(sample_n=5000, sample_random_state=666)

In [ ]:
# fit pyTorch input 
features = np.transpose(images, (0,3,1,2))
features.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.05, random_state=42)

### PyTorch dataset and loader

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# train
dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
loader = DataLoader(dataset, batch_size=100, shuffle=True)

In [ ]:
# test
X_test, y_test = torch.FloatTensor(X_test), torch.LongTensor(y_test)

# Model Training

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from efficientnet_pytorch import EfficientNet



class predictor_EfficientNet(nn.Module):
    def __init__(self, img_size = (224,224), n_classes=42):
        super().__init__()
        
        self.n_classes = n_classes
        self.img_size = img_size
        
        # load/freeze backbone
        self.backbone = None
        self._loadBackbone()
        self._freezeBackbone()
        
        # predictor
        ## b0
#         self._dropout = nn.Dropout(0.2)
#         self._fc = nn.Linear(1280, self.n_classes)
        
        ## b3
        self._avg_pooling = nn.AdaptiveAvgPool2d(1)
        self._dropout = nn.Dropout(0.3)
        self._fc = nn.Linear(1536, self.n_classes)
    
        ## b7
#         self._avg_pooling = nn.AdaptiveAvgPool2d(1)
#         self._dropout = nn.Dropout(0.5)
#         self._fc = nn.Linear(2560, self.n_classes)
        
        
    def forward(self, x):
        bs = x.shape[0] # batch size
        
        # feature extractor
        x = self.backbone.extract_features(x)
        
        # predictor
        x = self._avg_pooling(x)
        x = x.view(bs, -1)
        x = self._dropout(x)
        x = self._fc(x)
        
        return x
        
    def _loadBackbone(self):
        self.backbone = EfficientNet.from_pretrained('efficientnet-b3')
        
    def _freezeBackbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False
        
model = predictor_EfficientNet()

In [ ]:
from torch import optim
from torch import nn

# epoch
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=1e-3)
CE_loss = nn.CrossEntropyLoss()

# check GPU
gpu_mode = torch.cuda.is_available()
if gpu_mode:
    model.cuda()
    CE_loss = CE_loss.cuda()
    X_test, y_test = X_test.cuda(), y_test.cuda()

log = {'train_loss':[], 'valid_loss':[], 'valid_accuracy':[]}

for epoch in range(epochs):
    acc_loss = 0.
    model.train()
    for x, y in loader:
        if gpu_mode:
            x, y = x.cuda(), y.cuda()
        
        y_ = model(x)
        loss = CE_loss(y_, y)
        loss.backward()
        optimizer.step()
        
        acc_loss += loss.item()
    

    model.eval()
    # check loss    
    y_pred = model(X_test)
    val_loss = CE_loss(y_pred, y_test)
    # check accuracy
    _, pred = torch.max(y_pred, 1)
    accuracy = ((pred == y_test).sum()).item()/len(y_test)
    
    # log
    train_loss = acc_loss/len(dataset)
    valid_loss = val_loss/len(y_test)
    
    print('[%s] finish ! train loss : %.4f , valid loss : %.4f, valid accuracy : %.4f%%' % 
          (epoch, train_loss, valid_loss, 100 * accuracy))
    
    log['train_loss'].append(train_loss)
    log['valid_loss'].append(valid_loss)
    log['valid_accuracy'].append(accuracy)
        

# Submission prediction

In [ ]:
import pickle
torch.save(model.state_dict(), './model/EfficientNet/b3_v1.pkl')